In [1]:
from __future__ import print_function
import os
import keras
import tensorflow as tf

config = tf.ConfigProto( device_count = {'GPU': 1 , 'CPU': 56} ) 
sess = tf.Session(config=config) 
keras.backend.set_session(sess)

from custom_model.layers_keras import *
from custom_model.model_keras import *
from custom_model.math_utils import *
from utils_vis import *
from keras import metrics

import matplotlib.pyplot as plt
%matplotlib inline
import h5py
import numpy as np

Using TensorFlow backend.


In [2]:
dt = 2
horizon = 20
OBS = 30//dt-1
PRED = horizon//dt

model_name = 'dfn'

Data = h5py.File('Datasets/RRot_cc2_20.h5', 'r')
x_train = np.array(Data['Speed_obs_train'])
y_train = np.array(Data['Speed_pred_train'])
e_train = np.array(Data['E_train'])
x_test = np.array(Data['Speed_obs_test'])
y_test = np.array(Data['Speed_pred_test'])
e_test = np.array(Data['E_test'])

'''
x_test = x_test[:3600]
y_test = y_test[:3600]
e_test = e_test[:3600]

x_test = np.delete(x_test, np.s_[3150:3180], axis=0)
y_test = np.delete(y_test, np.s_[3150:3180], axis=0)
e_test = np.delete(e_test, np.s_[3150:3180], axis=0)

x_test = x_test[:1440]
y_test = y_test[:1440]
e_test = e_test[:1440]

x_test = np.delete(x_test, np.s_[1260:1272], axis=0)
y_test = np.delete(y_test, np.s_[1260:1272], axis=0)
e_test = np.delete(e_test, np.s_[1260:1272], axis=0)
'''
x_train = x_train[:,-OBS:]
y_train = y_train[:,:PRED]
e_train =e_train[:,:PRED]
y_test = y_test[:,:PRED]
e_test =e_test[:,:PRED]

In [ ]:
if model_name == 'dfn':
    model = create_embed_model(obs_timesteps=OBS, pred_timesteps=PRED, nb_nodes=208, k=3, dgc_mode='dgc',model_name='dfn')
elif model_name == 'dgcgru':
    model = create_embed_model(obs_timesteps=OBS, pred_timesteps=PRED, nb_nodes=208, k=2, dgc_mode='dgc',model_name='dgcgru')

In [3]:
encoder = RNN(RecurrentDGC(k=3, units=64), return_sequences=True, return_state=True)
decoder = RNN(RecurrentDGC(k=3, units=64), return_sequences=True, return_state=True)
        
#readout = Dense(208, activation='sigmoid')
unstack_k = Lambda(unstack)
choice = Scheduled()
    
input_obs = Input(shape=(OBS, 208, 1)) 
input_gt = Input(shape=(PRED, 208, 1)) #(None, 4, 208, 1)
encoder_inputs = Lambda(lambda x: K.squeeze(x, axis = -1))(input_obs) # (None, 29, 208)
    
encoder_outputs, state_h = encoder(encoder_inputs)
unstacked = unstack_k(input_gt) #[(None, 208, 1) x 4] list
    
initial = unstacked[0] #(None, 208, 1)
decoder_inputs = Lambda(lambda x: K.permute_dimensions(x, (0,2,1)))(initial) #(None, 1, 208)
decoder_outputs_new, state_h_new = decoder(decoder_inputs, initial_state=state_h)
state_h = state_h_new
#prediction = []
feature = []
#decoded_results = readout(decoder_outputs_new)
#prediction.append(decoded_results)
feature.append(decoder_outputs_new)
if PRED > 1:       
    for i in range(1,PRED):
        decoder_inputs = unstacked[i]#(None, 208, 1)
        decoder_inputs = Lambda(lambda x: K.permute_dimensions(x, (0,2,1)))(decoder_inputs)#(None, 1, 208)
        decoder_outputs_new, state_h_new = decoder(decoder_inputs, initial_state=state_h)
        state_h = state_h_new
        #decoded_results = readout(decoder_outputs_new)
        feature.append(decoder_outputs_new)
    
outputs = Lambda(stack)(feature)
model = Model([input_obs, input_gt], outputs)

In [4]:
#filename = 'load_models/cc_dggru-'+str(dt)+'-'+str(horizon)+'_k2.h5'
filename = 'load_models/cl_dgfn_dt2.h5'
file=h5py.File(filename,'r')
weight = []
for i in range(len(file.keys())):
    weight.append(file['weight'+str(i)][:])
model.set_weights(weight)

In [ ]:
y = model.predict([x_test,e_test])
for i in range(1,PRED+1):   
    print(MAP(y_test[:,:i]*120, y[:,:i]*120), ' ',100*MAPE(y_test[:,:i]*120, y[:,:i]*120), ' ',RMSE(y_test[:,:i]*120, y[:,:i]*120))
print('##########')
for i in range(PRED):
    print(MAP(y_test[:,i]*120, y[:,i]*120), ' ',100*MAPE(y_test[:,i]*120, y[:,i]*120), ' ',RMSE(y_test[:,i]*120, y[:,i]*120))

In [ ]:
day = 2
start = 150*(day-1)
end = start+150
j=3
gt = [y_test[i] for i in range(start,end,PRED)]
pred = [y[i][:j] for i in range(start,end,j)]

ground_truth = np.concatenate(gt, axis=0)
prediction = np.concatenate(pred, axis=0)

print(ground_truth.shape)
print(prediction.shape)

In [ ]:
plot_figure(ground_truth, title1 = 'speed ground-truth on Rot_cc, 15/01/2018 (dt=2min)', title2 = ' ', nb = 199, figtitle='test.PNG', time=150, dt=2/60, color='jet_r')

In [ ]:
plot_figure(prediction, title1 = '10 min prediction of DGGRU, k=2', title2 = ' ',nb = 208, figtitle='test.PNG',time=150,dt=2/60,color='jet_r')

In [5]:
y = model.predict([x_train,e_train])

In [6]:
W = y[...,1:]
W.shape

(16500, 2, 199, 199)

In [7]:
step = 0
W = W[:,[step]]
W.shape

(16500, 1, 199, 199)

In [ ]:
names = [weight.name for layer in model.layers for weight in layer.weights]
weights = model.get_weights()

for name, weight in zip(names, weights):
    print(name, weight.shape)

In [ ]:
A,B,C = Speed_k_correlation(ground_truth=120*e_train[:,[step]].squeeze(axis=-1), weights=W, k=3)

In [14]:
interval = 16500//4
start = 3*interval
end = start+interval
E = 120*e_train[:,[step]].squeeze(axis=-1)
A,B,C = Speed_k_correlation(ground_truth=E[start:end], weights=W[start:end], k=3)

[115, 110, 105, 100, 95, 90, 85, 80, 75, 70, 65, 60, 55, 50, 45, 40, 35, 30, 25, 20, 15, 10, 5, 0]
[  9498   2869   9708  94848 217268 143263  81729  44314  52046  24567
  15370  13523  12021  10980  12402  13771  12879  10873   9139   8303
   7366   6909   6013   1216]


In [ ]:
f = Compare(ground_truth=120*e_train[:,[step]].squeeze(), weights=W.squeeze(), k=3)

In [15]:
for i in range(len(A)):
    print(A[23-i])

1.4918829298542389
1.618662196086822
1.5683352740981844
1.498229727372614
1.4540897061251448
1.4443426447982732
1.464586654649521
1.4238072498184244
1.3228696468129253
1.2102664348577694
1.1817072201454852
1.0276381397483567
0.9371024439746466
0.8569835691301247
0.5648342123512355
0.3841574973123835
0.5188875680312486
-0.05351832058036121
-0.006326308362099959
0.39109668033037215
0.6634431527907145
0.3293065530923507
0.4060473105460365
0.797215134954234


In [ ]:
for i in range(len(C)):
    print(np.var(np.array(C[i])))

In [ ]:
plt.figure(figsize = (13,5))
labels = [i for i in range(0, 125, 5)]
y_lab = [-3.5,-2.5,-1.5,-0.5,0.5,1.5,2.5,3.5]
plt.xlabel("v(km/h)",fontsize=22)
plt.xlim(0,120)
plt.ylim(-3.5,3.5)
plt.tick_params(axis='both', which='major', labelsize=18)
plt.ylabel("Spatial importances J()",fontsize=22)
plt.pcolormesh(labels, y_lab, np.flip(B,axis=1), cmap = 'jet')
plt.title('Spatial Correlations vs speed on ROTCL, dt=5min', fontsize=20)
plt.colorbar()
plt.clim(0,0.35)
plt.show()

In [ ]:
data = [sorted(np.array(C[23-i])) for i in range(24)]
labels = [i for i in range(5, 125, 5)]
plt.figure(figsize=(18,8))
plt.xlabel("v(km/h)",fontsize=24)
plt.xlim(0,125)
plt.ylim(-3,3)
plt.tick_params(axis='both', which='major', labelsize=18)
plt.ylabel("attention coefficient f",fontsize=24)
plt.title("Attention coefficent distributions & mean values on ROTCL, dt=5min",fontsize=24)
pc = plt.violinplot(
        data, labels, widths=4., showmeans=True, showmedians=False,
        showextrema=False, bw_method='silverman')
plt.show()

In [ ]:
plt.savefig('Figs/violin2.PNG')

In [ ]:
head = 100*78
end = head+101
s = 199
gt = 120*y_train[:,[step]].squeeze()

plt.figure(figsize = (15,8))
y_lab = [0.2*i for i in range(s)]
x_lab = [14+3/60*i for i in range(101)]
#y, x = np.mgrid[slice(0, (s+1)*0.2, 0.2), slice(14-4/60., 14+150*2/60., 2/60)]
plt.xlabel("time (PM)",fontsize=22)
plt.xlim(14,19)
plt.ylim(0,(s-0.2)*0.2)
plt.tick_params(axis='both', which='major', labelsize=18)
plt.ylabel("position (km)",fontsize=22)
plt.pcolormesh(x_lab, y_lab, f[head:end].transpose(), cmap = 'coolwarm')
#levels1 = MaxNLocator(nbins=120).tick_values(-3, 3)
#plt.contourf(x[:-1, :-1] + 0.2/2.,
                  #y[:-1, :-1] + 2/120., [head:end].transpose(), levels=levels1,
                  #cmap='RdYlBu_r')
plt.title('Attention coefficient evolution on 13-11-2018 (dt=4min, ROT_CC)', fontsize=20)
plt.colorbar()
#plt.clim(0,1)
plt.show()